In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [265]:
import cPickle as pickle
from itertools import permutations, combinations
from fuzzywuzzy import fuzz, process
from project_modules.flavorNetworkBuilder.recipe_weights import compute_all_weights, compute_single_avg_weight
from project_modules.flavorNetworkBuilder.ingredient_matcher import IngredientMatcher
from project_modules.flavorNetworkBuilder.recommender import FlavorRecommender

## testing recommender

In [14]:
def get_range(dictionary, begin, end):
  return dict(itertools.islice(dictionary.iteritems(), begin, end+1)) 

In [4]:
data_dir = '../../data/'

In [5]:
file_name = 'ldc_recipes.pkl'
with open(data_dir+file_name, 'r') as f:
    ldc_recipes  = pickle.load(f)
    
file_name = 'nyt_recipes.pkl'
with open(data_dir+file_name, 'r') as f:
    nyt_recipes  = pickle.load(f)

In [6]:
all_recipes = {}
for cocktail in ldc_recipes:
    all_recipes.update(cocktail)
all_recipes.update(nyt_recipes)

In [7]:
file_name = 'second_ing_flav_dict.pkl'
with open(data_dir+file_name, 'r') as f:
    comp_ing_dict = pickle.load(f)

In [8]:
file_name = 'backbone_edges_dict.pkl'
with open(data_dir+file_name) as f:
    edge_dict = pickle.load(f)

In [9]:
file_name = 'full_edges_dict.pkl'
with open(data_dir+file_name) as f:
    full_edge_dict = pickle.load(f)

In [10]:
file_name = 'match_dict.pkl'
with open(data_dir+file_name) as f:
    match_dict = pickle.load(f)

In [15]:
# with open('./avg_recipe_weights.pkl', 'r') as f:
#     avg_w = pickle.load(f)
# with open('./new_avg_recipe_weights.pkl', 'r') as f:
#     new_avg_w = pickle.load(f)

# with open('./random_weights_new.pkl', 'r') as f:
#     random_weights = pickle.load(f)

# with open('./new_avg_recipe_weights.pkl', 'r') as f:
#     avg_recipe = pickle.load(f)
    
# with open('./new_avg_random_weights.pkl', 'r') as f:
#     avg_random = pickle.load(f)

In [16]:
cocktail_ing_lists = all_recipes.values()
cocktail_set = set([i.strip().lower() for sublist in cocktail_ing_lists for i in sublist if i])

In [17]:
len(cocktail_set)

1442

In [39]:
match_dict = {k:match_ingredients([k], comp_ing_dict) for k in unique_ing_list}

In [ ]:
%prun {k:match_ingredients([k], comp_ing_dict) for k in unique_ing_list}

In [59]:
with open('./match_dict.pkl', 'w')  as f:
    pickle.dump(match_dict, f)

In [96]:
small_match_dict  = {k:match_ingredients([k], comp_ing_dict) for k in ['rye whiskey', 'orange']}

In [275]:
%page all_recipes

In [189]:
cocktail_set_list = list(cocktail_set)

In [97]:
filter_list = ['chocolate bitters', u'pineapple', 'pineapple puree', 'apple','ginger beer','peach bitters','mango nectar','angostura bitters']

In [344]:
# ingred_opt(all_recipes['Elderflower Martini'])

In [369]:
ingred_opt(new_drink)

(u'cardamom bitters', 216.0)

In [128]:
comp_ing_dict['wormwood']

[]

In [124]:
matcher.match_ingredients(['wormwood'])

[]

In [130]:
matcher = IngredientMatcher(comp_ing_dict)
matcher.match_ingredients(['campari'])

[u'cloves',
 u'angelica root oil',
 u'clove',
 u'anise',
 u'calamus',
 u'wormwood',
 u'rosemary',
 u'lemon peel',
 u'sage',
 u'thyme',
 u'cala-mus',
 u'the bitter orange',
 u'fennel',
 u'marjoram',
 u'bitter orange peel oil',
 u'orange (bitter) peel oil',
 u'cin-namon',
 u'bitter orange peel',
 u'cinnamon']

In [159]:
def get_range(dictionary, begin, end):
  return dict(itertools.islice(dictionary.iteritems(), begin, end+1)) 
sub_rec = get_range(all_recipes, 0, 100)

In [166]:
my_rec = FlavorRecommender(comp_ing_dict, edge_dict, all_recipes, True)

In [170]:
with open('match_dict.pkl', 'r') as f:
    pre_match_dict = pickle.load(f)

In [172]:
# with open('match_dict.pkl', 'w') as f:
#     pickle.dump(my_rec.match_dict, f)

In [158]:
len(all_recipes)

1585

In [184]:
all_recipes

{u'The Battle of Ivrea': [u'lime',
  u'lemon',
  u'orange',
  u'Hella Bitter citrus bitters',
  u'light rum',
  u'Campari',
  u'Moscato'],
 u'Happy Holidaze': [u'hot chocolate', u'green Chartreuse', u'Cointreau'],
 u'Grapefruit Negroni': [u'Campari',
  u'grapefruit peel',
  u'ruby red grapefruit',
  u'ruby red grapefruit juice',
  u'sweet vermouth',
  u'Gin,'],
 u'Rob Roy': [u'Angostura Bitters', u'cherry', u'sweet vermouth', u'Scotch,'],
 u'The Translucent Cocktail': [u'cherry',
  u'cr\xe8me de cacao',
  u'kirschwasser',
  u'Gin'],
 u'Piha Spritz Recipe': [u'Fresh lime juice',
  u'Fresh pineapple juice',
  u'herbal tea bag',
  u'palm sugar',
  u'sage flower',
  u'white sugar',
  u'Gin,'],
 u'Scarlet Glow': [u'grapefruit juice',
  u'hibiscus tea',
  u'sugar',
  u'Yellow Chartreuse',
  u'Pisco'],
 u'Spiced Pumpkin Flip': [u'agave nectar',
  u'amaretto',
  u'pumpkin ale',
  u'pumpkin butter',
  u'whole egg',
  u'Bourbon'],
 u'Orange Poppy': [u'Lemon-lime soda',
  u'mint',
  u'orange',
  

In [329]:
new_drink = ['gin' ,'lemon', 'cucumber']
new_rec.make_rec(new_drink, 10)

u'white tea liqueur'

In [325]:
new_rec = FlavorRecommender(comp_ing_dict, edge_dict, all_recipes, pre_match_dict)

NameError: global name 'drink_recipe' is not defined

In [273]:
for recipe in all_recipes.keys()[:100]:
#     print all_recipes[recipe]
    print new_rec.make_rec_2(all_recipes[recipe])
    print '\n'

(u'victoria beer', 76.0)


(u'coco lopez cream of coconut', 7.0)


(u'mango rum', 52.0)


(u'coco lopez cream of coconut', 7.0)


(u'mango rum', 45.0)


(u'victoria beer', 59.0)


(u'victoria beer', 59.0)


(u'aj stephans ginger beer', 21.0)


(u'coco lopez cream of coconut', 7.0)


(u'ardbeg whisky', 12.0)


(u'aj stephans ginger beer', 21.0)


(u'victoria beer', 28.0)


(u'coco lopez cream of coconut', 11.0)


(u'ardbeg whisky', 24.0)


(u'ardbeg whisky', 32.0)


(u'coco lopez cream of coconut', 10.0)


(u'aj stephans ginger beer', 48.0)


(u'coco lopez cream of coconut', 10.0)


(u'mango rum', 45.0)


(u'coco lopez cream of coconut', 7.0)


(u'mango rum', 45.0)


(u'ardbeg whisky', 12.0)


(u'aj stephans ginger beer', 21.0)


(u'mango rum', 45.0)


(u'ardbeg whisky', 6.0)


(u'mango rum', 45.0)


('no suggestion', 0)


(u'coco lopez cream of coconut', 15.0)


(u'coco lopez cream of coconut', 14.0)


(u'coco lopez cream of coconut', 11.0)


(u'mango rum', 45.0)


(u'coco lopez cream 

KeyboardInterrupt: 

In [ ]:
new_rec.make_rec2